In [21]:
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt


import torch
import torch.nn as nn
import torch.nn.functional as F

import train_test 

from load_dataset import MyOwnDatasetFixedKnn
from models import GAT, GCN, GraphSAGE

In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


# Variando numero de epocas

In [24]:
from torch_geometric.loader import DataLoader

path1 = "TF_encodec_flatten_signal_t4.csv"
path2 = "UrbanSound8K_t4_6864.hdf5"
dataset = MyOwnDatasetFixedKnn(root = "../own_dataset/data_encodec_flatten_fixed_knn", path = [path1,path2])
loader_dataset = DataLoader(dataset)

In [25]:
for i in dataset:
    print(i)

Data(x=[6864, 38400], edge_index=[2, 34320], y=[6864], train_mask=[5461], test_mask=[718], val_mask=[685])
Data(x=[6864, 38400], edge_index=[2, 34320], y=[6864], train_mask=[5426], test_mask=[685], val_mask=[753])
Data(x=[6864, 38400], edge_index=[2, 34320], y=[6864], train_mask=[5365], test_mask=[753], val_mask=[746])
Data(x=[6864, 38400], edge_index=[2, 34320], y=[6864], train_mask=[5380], test_mask=[746], val_mask=[738])
Data(x=[6864, 38400], edge_index=[2, 34320], y=[6864], train_mask=[5489], test_mask=[738], val_mask=[637])
Data(x=[6864, 38400], edge_index=[2, 34320], y=[6864], train_mask=[5544], test_mask=[637], val_mask=[683])
Data(x=[6864, 38400], edge_index=[2, 34320], y=[6864], train_mask=[5589], test_mask=[683], val_mask=[592])
Data(x=[6864, 38400], edge_index=[2, 34320], y=[6864], train_mask=[5605], test_mask=[592], val_mask=[667])
Data(x=[6864, 38400], edge_index=[2, 34320], y=[6864], train_mask=[5552], test_mask=[667], val_mask=[645])
Data(x=[6864, 38400], edge_index=[2, 

In [26]:
i.y.dtype

torch.int64

In [14]:
#GAT

  
prom_acc_gat = []
dev_values_gat = []
num_epochs_gat = []

#GCN


prom_acc_gcn = []
dev_values_gcn = []
num_epochs_gcn = []

#SAGE


prom_acc_sage = []
dev_values_sage = []
num_epochs_sage = []

In [27]:
for i in range(30,2230,200):
#GAT

  acc_gat = []
 

#GCN

  acc_gcn = []


#SAGE

  acc_sage = []


  for n, graphs in enumerate(loader_dataset):
      
#==========================GAT===============================================
      graphs.to(device)
      gat = GAT(38400, 20, 10).to(device)
      train_test.train(gat, graphs, epoch = i, enable = False)
      acc_gat.append(train_test.test(gat, graphs, graphs.test_mask))

#==========================GCN===============================================  

      
      gcn = GCN(38400, 20, 10).to(device)
      train_test.train(gcn, graphs, epoch = i, enable = False)
      acc_gcn.append(train_test.test(gcn, graphs, graphs.test_mask))

#==========================SAGE=============================================== 

      
      g_sage = GraphSAGE(38400, 20, 10).to(device)
      train_test.train(g_sage, graphs, epoch = i, enable = False)
      acc_sage.append(train_test.test(g_sage, graphs, graphs.test_mask))

#==========================GAT===============================================

  prom_acc_gat.append(np.mean(acc_gat))
  dev_values_gat.append(np.std(acc_gat))
  num_epochs_gat.append(i)

#==========================GCN===============================================  

  prom_acc_gcn.append(np.mean(acc_gcn))
  dev_values_gcn.append(np.std(acc_gcn))
  num_epochs_gcn.append(i)

#==========================SAGE===============================================  

  prom_acc_sage.append(np.mean(acc_sage))
  dev_values_sage.append(np.std(acc_sage))
  num_epochs_sage.append(i)

  print(f"GAT\n accuracy mean: {np.mean(acc_gat)*100:.2f}% | standard deviation: {np.std(acc_gat):.2f} | epochs: {i}\n"
        f"GCN\n accuracy mean: {np.mean(acc_gcn)*100:.2f}% | standard deviation: {np.std(acc_gcn):.2f} | epochs: {i}\n"
        f"SAGE\n accuracy mean: {np.mean(acc_sage)*100:.2f}% | standard deviation: {np.std(acc_sage):.2f} | epochs: {i}\n\n"
        f"==============================================================================================================")

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.96 GiB (GPU 0; 5.81 GiB total capacity; 2.33 GiB already allocated; 1.22 GiB free; 3.99 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
arr_num_epochs_gat = np.array(num_epochs_gat)
arr_prom_acc_gat = np.array(prom_acc_gat)
arr_dev_values_gat = np.array(dev_values_gat)


arr_num_epochs_gcn = np.array(num_epochs_gcn)
arr_prom_acc_gcn = np.array(prom_acc_gcn)
arr_dev_values_gcn = np.array(dev_values_gcn)

arr_num_epochs_sage = np.array(num_epochs_sage)
arr_prom_acc_sage = np.array(prom_acc_sage)
arr_dev_values_sage = np.array(dev_values_sage)

In [ ]:
f = h5py.File("./variacion_epochs_encodec_flatten.hdf5", "w")

dset = f.create_dataset("Numero_epochs_GAT", arr_num_epochs_gat.shape, data = arr_num_epochs_gat)
dset = f.create_dataset("Promedios_Acc_GAT", arr_prom_acc_gat.shape, data = arr_prom_acc_gat)
dset = f.create_dataset("desviacion_GAT", arr_dev_values_gat.shape, data = arr_dev_values_gat)

dset = f.create_dataset("Numero_epochs_GCN", arr_num_epochs_gcn.shape, data = arr_num_epochs_gcn)
dset = f.create_dataset("Promedios_Acc_GCN", arr_prom_acc_gcn.shape, data = arr_prom_acc_gcn)
dset = f.create_dataset("desviacion_GCN", arr_dev_values_gcn.shape, data = arr_dev_values_gcn)

dset = f.create_dataset("Numero_epochs_SAGE", arr_num_epochs_sage.shape, data = arr_num_epochs_sage)
dset = f.create_dataset("Promedios_Acc_SAGE", arr_prom_acc_sage.shape, data = arr_prom_acc_sage)
dset = f.create_dataset("desviacion_SAGE", arr_dev_values_sage.shape, data = arr_dev_values_sage)


f.close()